# 1. EDA

## データセット読み込み

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import japanize_matplotlib
import seaborn as sns
# from scipy.stats import norm    #確率分布関連
# from scipy import stats    #統計関連
# from scipy.special import boxcox1p
from scipy import stats
from scipy import special
%matplotlib inline

pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 500)

df_train = pd.read_csv("train_test_submission/train.csv", index_col=0)
df_test = pd.read_csv("train_test_submission/test.csv", index_col=0)

print(f"df_train.shape: {df_train.shape}")
display(df_train.head(3))
print(f"df_train.shape: {df_test.shape}")
display(df_test.head(3))

## ydata_profilingを使う場合（普段はコメントアウト。時間かかるので注意）

In [ ]:
# # ydata_profilingを使う場合

# from ydata_profiling import ProfileReport
# profile = ProfileReport(df_train, minimal=True)
# profile.to_file("ydata_profiling/kaggle_houseprices.html")

## 目的変数(SalePrice)の特徴把握

In [ ]:
data = df_train["SalePrice"]

# 目的変数（SalePrice）の基本統計量を表示
print("-" * 10, "describe", "-" * 10)
print(round(data.describe(), 1))

# 歪度と尖度
skewness = data.skew()
kurtosis = data.kurtosis()
print(f"歪度(Skewness: {skewness})")
print(f"尖度(Kurtosis: {kurtosis})")

# グラフ描写
fig, ax = plt.subplots(1, 2, figsize=(10, 4))

sns.histplot(data, stat="density", kde=True, ax=ax[0])
ax[0].set_title("ヒストグラムと正規分布")
ax[0].tick_params(axis="x", labelsize=8)  # 軸の文字サイズ変更

# 正規分布を重ねて表示（黒線）
xmin, xmax = ax[0].get_xlim()
x = np.linspace(xmin, xmax, 100)
p = stats.norm.pdf(x, np.mean(data), np.std(data))
ax[0].plot(x, p, "k", linewidth=1)

# 正規確率プロットを表示するサブプロット
res = stats.probplot(data, plot=ax[1])
ax[1].set_title("正規確率プロット")

plt.tight_layout()
plt.show()

# ・歪度 (skewness) の絶対値が 1.88 と高い
# ・ヒストグラムの正規分布線 (黒線) と KDE のズレが発生
# ・QQプロット線 (青線) が正規分布線 (赤線) からズレが発生
# 以上３点からSalePriceが正規分布に対して歪んでいることが分かる。
# 正規分布から歪んでいると、機械学習の精度の低下につながる。

## SalePriceと他の特徴量との関係

In [ ]:
# GrLivArea	地上（地上）の生活エリアの平方フィート。
ax = sns.scatterplot(data=df_train, x="GrLivArea", y="SalePrice")

# 右下に2つ大きく傾向から外れた値がある。
# この２点は住居エリアが広くて、住宅価格が安い
# おそらく郊外のエリア。外れ値の候補として考えておく

In [ ]:
# YearBuilt 建築年
plt.figure(figsize=(16, 5))
ax = sns.boxplot(data=df_train, x="YearBuilt", y="SalePrice")
plt.xticks(rotation=90)
plt.show()
# 意外とあまり傾向が見えない？

In [ ]:
# OverallQual：	全体的な材料と仕上げの品質。
ax = sns.boxplot(data=df_train, x="OverallQual", y="SalePrice")

## 相関係数

In [ ]:
# すべての特徴量について
plt.figure(figsize=(12, 10))
sns.heatmap(
    df_train.corr(numeric_only=True),
    cmap="viridis",
    annot=True,
    fmt=".1f",
    annot_kws={"fontsize": 6},
)

In [ ]:
# SalePriceとの相関がある特徴量のみについて

threshold = 0.4  # 相関係数の閾値
high_corr_cols = (
    df_train.corr(numeric_only=True)["SalePrice"][
        abs(df_train.corr(numeric_only=True)["SalePrice"]) >= threshold
    ]
    .sort_values(ascending=False)
    .index
)

sns.heatmap(
    df_train[high_corr_cols].corr(),
    cmap="viridis",
    annot=True,
    fmt=".1f",
    annot_kws={"fontsize": 10},
)
plt.show()

# 方針：特徴量同士で相関係数が高い項目に関しては、片方を除外、もしくは、別の特徴量を検討
# GarageCarsが多ければ、GarageAreaが多いのも必然のためGarageAreaは除外
# GrLivArea(地上の生活エリア)が広ければ、TotRmsAbvGrd(地上の部屋数)も多くなるのが一般的なので、TotRmsAbvGrdは除外
# 車庫と建物が同時に建設されることが一般的なので、YearBuiltとGarageYrBltが相関が高くなっていると考えられるため、GarageYrBitは除外

In [ ]:
# scatterplot
threshold = 0.6  # 相関係数の閾値
high_corr_cols = (
    df_train.corr(numeric_only=True)["SalePrice"][
        abs(df_train.corr(numeric_only=True)["SalePrice"]) >= threshold
    ]
    .sort_values(ascending=False)
    .index
)

sns.set(
    font="IPAexGothic"
)  # ここでフォントを指定しておかないと、後のグラフ描画にて日本語が文字化けする
sns.pairplot(df_train[high_corr_cols], height=2.5)
plt.show()

# 2. 前処理

## 欠損値処理

In [ ]:
# DataFrameを結合
df_all_data = pd.concat([df_train, df_test])

# 欠損値の数を計算
missing_values_count = df_all_data.isna().sum()
missing_values_table = pd.DataFrame(
    {
        "Missing_total": missing_values_count,
        "Percent (%)": round(missing_values_count / len(df_all_data) * 100, 2),
    }
)

# 欠損値の割合順で並べ替え
df_missing = missing_values_table[
    missing_values_table["Missing_total"] > 0
].sort_values(by="Missing_total", ascending=False)
# 処理法を決めるのに使う作業用csvを出力
df_data_description = pd.read_csv(
    "data_description/data_description_日本語.csv", index_col=0
)
df_missing_value_processing = pd.concat([df_missing, df_data_description], axis=1)
# encoding="utf-8_sig"を付けることでexcelで開いたときの文字化けを回避
df_missing_value_processing.to_csv(
    "missing_value_processing/missing_value_processing_欠損値処理.csv",
    encoding="utf-8_sig",
)

display(df_missing)

In [ ]:
# "LotFrontage"について
plt.figure(figsize=(20, 4))
sns.boxplot(data=df_all_data, x="Neighborhood", y="LotFrontage")
plt.show()
# 下の結果から、地域ごとに"LotFrontage"の値にばらつきがあることが分かる

In [ ]:
# 各地域"Neighborhood"の"LotFrontage"の中央値
df_group_LotFrontage = df_all_data.groupby(by="Neighborhood")["LotFrontage"].agg(
    "median"
)


def fillnaLot(row):
    """
    ある1つの住宅データについて、"LotFrontage"列の値が欠損している場合はそのデータの地域（"Neighborhood"）の"LotFrontage"の中央値で補完。
    欠損していない場合、元の値をそのまま返す。

    Parameter
    ----------
    row : pandas.core.frame.DataFrame
        欠損値処理をしたいデータセット

    Return
    ----------
    欠損値処理がされたLotFrontage行が返ってくる
    """
    # もし"LotFrontage"が欠損値（NaN）または空文字列の場合
    # NaN	「何もないよー」な意味をあらわす単語
    # 空文字列	長さ0文字の文字列のこと
    # でもisnaは空文字列""を欠損値として認識するはずだけど…？取り敢えず教材通りに書いておく

    if pd.isna(row["LotFrontage"]) or row["LotFrontage"] == "":
        return df_group_LotFrontage[row["Neighborhood"]]
    else:
        return row["LotFrontage"]

In [ ]:
######## ミュータブル操作とイミュータブル操作 #########

# イミュータブル（不変）とミュータブル（可変）の操作の違いが関係しています。以下の例を使って詳細を説明します。

# 例1: ミュータブル操作
# df[col] = df[col].fillna("None")は、データフレームの列を直接変更します。この場合、データフレームの列はミュータブルなオブジェクトであり、変更は元のデータフレームに反映されます。

# 例2: イミュータブル操作
# df = df.drop('Utilities', axis=1)は、元のデータフレームを変更せずに、新しいデータフレームを作成してdfに代入します。この場合、元のデータフレームは変更されません。

# もう少し具体的に比較すると：

# ミュータブル操作（列の変更）
# for df in datasets:
#     for col in cols_fillNone:
#         df[col] = df[col].fillna("None")
# このループでは、df[col]はdatasets内の各データフレームの特定の列を参照しており、fillnaを適用してその列を更新しています。この変更はdatasetsの元のデータフレームに反映されます。

# イミュータブル操作（データフレーム全体の置換）
# for i in range(len(datasets)):
#     datasets[i] = datasets[i].drop('Utilities', axis=1)
# このループでは、dropメソッドは新しいデータフレームを生成します。datasets[i]をその新しいデータフレームに更新することで、元のデータフレームも更新されます。

# したがって、以下のようにインデックスを使って元のデータフレームを更新すれば、Utilities列を削除する操作も期待通りに動作します。

# for i in range(len(datasets)):
#     if 'Utilities' in datasets[i].columns:
#         datasets[i] = datasets[i].drop('Utilities', axis=1)

In [ ]:
datasets = [df_train, df_test]

# LotFrontageの穴埋め
for df in datasets:
    df["LotFrontage"] = df.apply(fillnaLot, axis=1)

# Noneで穴埋めするもの
cols_fillNone = [
    "PoolQC",
    "MiscFeature",
    "Alley",
    "Fence",
    "FireplaceQu",
    "GarageCond",
    "GarageQual",
    "GarageFinish",
    "GarageType",
    "BsmtCond",
    "BsmtExposure",
    "BsmtQual",
    "BsmtFinType1",
    "BsmtFinType2",
]
# 0で穴埋めするもの
cols_fill0 = [
    "GarageYrBlt",
    "MasVnrArea",
    "BsmtFullBath",
    "BsmtHalfBath",
    "GarageCars",
    "GarageArea",
    "TotalBsmtSF",
    "BsmtUnfSF",
    "BsmtFinSF2",
    "BsmtFinSF1",
]
# 最頻値で穴埋めをするもの
cols_fillmode = [
    "MasVnrType",
    "MSZoning",
    "Functional",
    "Electrical",
    "KitchenQual",
    "Exterior2nd",
    "Exterior1st",
    "SaleType",
]

# ミュータブルとイミュータブルな操作が混じってるけど、まとめて対応するために以下のfor文を構築した
for i in range(len(datasets)):
    for col in cols_fillNone:
        datasets[i][col] = datasets[i][col].fillna("None")

    for col in cols_fill0:
        datasets[i][col] = datasets[i][col].fillna(0)

    for col in cols_fillmode:
        datasets[i][col] = datasets[i][col].fillna(datasets[i][col].mode()[0])

    # UtilitiesはAllPubが99.9%であり分析に使えないので列を削除
    if "Utilities" in datasets[i].columns:
        datasets[i] = datasets[i].drop("Utilities", axis=1)

In [ ]:
# 欠損値が補完できたことを確認するため、もう一度欠損値割合を出力する

# df_train, df_testの内容を更新する
df_train, df_test = datasets  # アンパック

# DataFrameを結合
df_all_data = pd.concat([df_train, df_test])

# 欠損値の数を計算
missing_values_count = df_all_data.isna().sum()
missing_values_table = pd.DataFrame(
    {
        "Missing_total": missing_values_count,
        "Percent (%)": round(missing_values_count / len(df_all_data) * 100, 2),
    }
)

# 欠損値の割合順で並べ替え
df_missing = missing_values_table[
    missing_values_table["Missing_total"] > 0
].sort_values(by="Missing_total", ascending=False)

display(df_missing)

# SalePrice以外の欠損値が全て消えており、補完が上手く言ったことが分かる

## 外れ値の取り扱い

In [ ]:
# GrLivArea	地上（地上）の生活エリアの平方フィート。
ax = sns.scatterplot(data=df_train, x="GrLivArea", y="SalePrice")

In [ ]:
# 右下の2点は面積が広いのに価格がとても安い。恐らく郊外のエリアであり、予測モデルにおいては傾向を邪魔すると考えられるため除外する
del_index = df_train.nlargest(2, "GrLivArea").index
print(f"before deleting: {df_train.shape}")
df_train = df_train.drop(del_index, axis=0)
print(f"after deleting: {df_train.shape}")
ax = sns.scatterplot(data=df_train, x="GrLivArea", y="SalePrice")
# グラフからも、右下の外れ値が除去できていることが確認できる

## 特徴量エンジニアリング

### 新しい特徴量'TotalSF'の作成

In [ ]:
# 新しい特徴量（'TotalSF'：'TotalBsmtSF'、'1stFlrSF'、'2ndFlrSF'を合計したもの）の作成
datasets = [df_train, df_test]

for df in datasets:
    print(f"Before making: {df.shape}")
    df["TotalSF"] = df["TotalBsmtSF"] + df["1stFlrSF"] + df["2ndFlrSF"]
    print((f"After making: {df.shape}\n"))

# それぞれ、特徴量が1つずつ増えていることが分かる
# 教材だとこの時点で特徴量が243個とかになっている…多分それ、one-hot encodingした後の値になってないかなぁ…

## 目的変数（SalePrice）の分布の調整

In [ ]:
data = df_train["SalePrice"]

# 調整前の状態を再度確認
print("-" * 10, "describe", "-" * 10)
print(round(data.describe(), 1))

# 歪度と尖度
skewness = data.skew()
kurtosis = data.kurtosis()
print(f"歪度(Skewness: {skewness})")
print(f"尖度(Kurtosis: {kurtosis})")

fig, ax = plt.subplots(1, 2, figsize=(10, 4))

sns.histplot(data, stat="density", kde=True, ax=ax[0])
ax[0].set_title("ヒストグラムと正規分布（調整前）")
ax[0].tick_params(axis="x", labelsize=8)

xmin, xmax = ax[0].get_xlim()
x = np.linspace(xmin, xmax, 100)
p = stats.norm.pdf(x, np.mean(data), np.std(data))
ax[0].plot(x, p, "k", linewidth=1)

res = stats.probplot(data, plot=ax[1])
ax[1].set_title("正規確率プロット（調整前）")

plt.tight_layout()
plt.show()

In [ ]:
# SalePriceに対数変換を適用し、歪度・ヒストグラム・QQプロットを確認

data = np.log(df_train["SalePrice"])

print("-" * 10, "describe", "-" * 10)
print(round(data.describe(), 1))

skewness = data.skew()
kurtosis = data.kurtosis()
print(f"歪度(Skewness: {skewness})")
print(f"尖度(Kurtosis: {kurtosis})")

fig, ax = plt.subplots(1, 2, figsize=(10, 4))

sns.histplot(data, stat="density", kde=True, ax=ax[0])
ax[0].set_title("ヒストグラムと正規分布（調整後）")
ax[0].tick_params(axis="x", labelsize=8)

xmin, xmax = ax[0].get_xlim()
x = np.linspace(xmin, xmax, 100)
p = stats.norm.pdf(x, np.mean(data), np.std(data))
ax[0].plot(x, p, "k", linewidth=1)

res = stats.probplot(data, plot=ax[1])
ax[1].set_title("正規確率プロット（調整後）")

plt.tight_layout()
plt.show()

# 結果、良さそうなのでdf_train["SalePrice"]を更新
df_train["SalePrice"] = data

## カテゴリカル変数のエンコーディング1
順番が意味を成す特徴量について（マップベースのカテゴリーエンコーディング）

In [ ]:
# 順番が意味を成すデータ品質(*Qual)・状態(*Cond)等には、マップベースのカテゴリーエンコーディングを適用していきます
# これめっちゃ面倒だけど…本当にこんなに手動でやってやらないといかんのか？特徴量が多いから面倒なのは仕方ないのか？
# 面倒なのは、カテゴリ間に順序があるものとないもので分けてエンコーディングをしているため。全部one-hotエンコーディングするとどうなる？

datasets = [df_train, df_test]
# 以下では、ラベル付の種類ごとにfor文を作って回しているが、何かもっとスマートに書けないか？

# 'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'HeatingQC', 'KitchenQual', 'FireplaceQu', 'GarageQual', 'GarageCond', 'PoolQC'について
# これらは'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, 'None': 0でラベル付け
label_mapping = {"Ex": 5, "Gd": 4, "TA": 3, "Fa": 2, "Po": 1, "None": 0}
cols = [
    "ExterQual",
    "ExterCond",
    "BsmtQual",
    "BsmtCond",
    "HeatingQC",
    "KitchenQual",
    "FireplaceQu",
    "GarageQual",
    "GarageCond",
    "PoolQC",
]
for i in range(len(datasets)):
    for col in cols:
        datasets[i][col] = datasets[i][col].map(label_mapping).astype(int)

# 'BsmtExposure'について
# これらは'Gd': 4, 'Av': 3, 'Mn': 2, 'No': 1, 'None': 0でラベル付け
label_mapping = {"Gd": 4, "Av": 3, "Mn": 2, "No": 1, "None": 0}
cols = ["BsmtExposure"]
for i in range(len(datasets)):
    for col in cols:
        datasets[i][col] = datasets[i][col].map(label_mapping).astype(int)

# 'GarageFinish'について
# これらは'Fin': 3, 'RFn': 2, 'Unf': 1, 'None': 0でラベル付け
label_mapping = {"Fin": 3, "RFn": 2, "Unf": 1, "None": 0}
cols = ["GarageFinish"]
for i in range(len(datasets)):
    for col in cols:
        datasets[i][col] = datasets[i][col].map(label_mapping).astype(int)

# 'BsmtFinType1', 'BsmtFinType2'について
# これらは'GLQ': 6, 'ALQ': 5, 'BLQ': 4, 'Rec': 3, 'LwQ': 2, 'Unf': 1, 'None': 0でラベル付け
label_mapping = {"GLQ": 6, "ALQ": 5, "BLQ": 4, "Rec": 3, "LwQ": 2, "Unf": 1, "None": 0}
cols = ["BsmtFinType1", "BsmtFinType2"]
for i in range(len(datasets)):
    for col in cols:
        datasets[i][col] = datasets[i][col].map(label_mapping).astype(int)

## 各特徴量（説明変数）の分布の調整

In [ ]:
# 各特徴量の分布についても歪度調整を行う。目的変数SalePriceについては対数変換を行ったが、こっちではBox-Cox変換を行う

# データ分析において、対数変換（`np.log`）と Box-Cox 変換（`scipy.special.boxcox1p`）は、データの分布を正規分布に近づけるために使用されるが、それぞれ異なる特徴と利点があります。

# ### 対数変換（`np.log`）
# - **方法**: 自然対数を取る変換。
#   ```python
#   np.log(x)
#   ```
# - **特性**:
#   - 数値が正でなければならない（ゼロや負の数値に対しては定義されない）。
#   - 変換後のデータが非常に小さい範囲になるため、元のデータのスケールが大きく影響する。
#   - データの分布が正のスキューを持つ場合に有効。

# ### Box-Cox 変換（`scipy.special.boxcox1p`）
# - **方法**: パラメータ λ（lambda）に基づく、非線形なパワー変換。
#   ```python
#   from scipy.special import boxcox1p
#   boxcox1p(x, λ)
#   ```
# - **特性**:
#   - λ パラメータにより、変換の形状を調整できるため、データに応じて最適な変換を見つけやすい。
#   - x が非負（ゼロを含む）であれば使用可能。
#   - λ = 0 のとき、`boxcox1p(x, λ)` は `np.log1p(x)`（x+1 の対数変換）と同等。
#   - より広い範囲のデータ分布を正規分布に近づけることができる。

# ### 違いのまとめ
# - **使用範囲**:
#   - `np.log`: 正のデータに限定される。
#   - `boxcox1p`: 非負のデータに対応。
# - **柔軟性**:
#   - `np.log`: 固定の変換方法。
#   - `boxcox1p`: λ パラメータにより柔軟な調整が可能。
# - **スキュー調整**:
#   - `np.log`: 主に正のスキューを持つデータに対して効果的。
#   - `boxcox1p`: λ の調整により、より多様な分布のデータに対応可能。

In [ ]:
# それぞれの特徴量の歪度を確認する

# データの結合
ntrain = len(df_train)
df_all_data = pd.concat([df_train, df_test])

# 数値変数（カテゴリ変数ではなく）であるものを特定
# https://note.nkmk.me/python-pandas-select-dtypes/#include pandas.DataFrameから特定の型の列を抽出・除外するselect_dtypesについて
numeric_feats = df_all_data.select_dtypes(include="number").columns

# 各数値変数の歪度を計算し、降順にソート。そしてその中で特に歪度が大きい（abs > 0.75）ものを取り出す
skew_bfBoxCox = (
    df_all_data[numeric_feats]
    .apply(lambda x: stats.skew(x.dropna()), axis=0)
    .sort_values(ascending=False)
)
skew_bfBoxCox = skew_bfBoxCox[abs(skew_bfBoxCox) > 0.75]

print(
    f"There are {skew_bfBoxCox.shape[0]} skewed numerical features to Box Cox transform (abs > 0.75)"
)
print(skew_bfBoxCox)

In [ ]:
# Box-Cox変換をしていく
skewed_index = skew_bfBoxCox.index
lam = 0.15  # Box-Cox変換におけるパラメータλを設定（天下り式に教材の値を設定）→本来ここは各特徴量毎に適切なλを設定する必要あり？
skew_aftBoxCox = pd.Series(index=skewed_index)

# 各数値変数に対してBox-Cox変換を適用し、列の値を更新
for idx in skewed_index:
    df_all_data[idx] = special.boxcox1p(df_all_data[idx], lam)
    skew_aftBoxCox[idx] = stats.skew(df_all_data[idx])

pd.DataFrame({"Skew bf Box-Cox": skew_bfBoxCox, "Skew aft Box-Cox": skew_aftBoxCox})

# Box-Cox変換を適用したことにより、概ね歪度が改善した
# だが、一部Box-Cox変換によって歪度が増大しているものもある。明らかに不適切だが、とりあえず教材通りこのまま進める

## カテゴリカル変数のエンコーディング2
順番が意味を成さない特徴量について（One-hot encoding）

In [ ]:
# 現時点ではまだdf_trainとdf_testはdf_all_dataの値に更新されていない。なので初めはdf_all_dataを使いながら作業を進めていく

# トレーニングデータの行数を取得
ntrain = len(df_train)

# one-hot encodingを実行
df_all_data = pd.get_dummies(df_all_data)

# df_all_dataを分割し、df_trainとdf_testに格納して中身を更新
df_train = df_all_data[:ntrain]
df_test = df_all_data[ntrain:]

# これまでの操作により、df_testの方に"SalePrice"行が入ってしまっているのでこれを削除
df_test = df_test.drop(["SalePrice"], axis=1)

# データの様子を表示
display(df_train)
display(df_test)
# 教材よりも行数が3つ少ない…何故？

# 3. モデルの選択と構築

## 使用する機械学習アルゴリズムの選定

In [ ]:
# 今回選定したアルゴリズムと特徴

# LinearRegression: 線形回帰モデルは、線形関係を仮定し、特徴量と目的変数との線形関係を学習します。最小二乗法を使用してモデルを適合させ、回帰係数を推定します。特徴としては、線形関係がある場合に効果的です。
# Ridge: リッジ回帰は線形回帰の一種で、L2正則化を使用して過学習を抑制することを特徴とします。正則化項は回帰係数の大きさを制約します。α（alpha）は正則化の強度を調整するハイパーパラメータです。
# Lasso: ラッソ回帰は線形回帰の一種で、L1正則化を使用して過学習を抑制することを特徴とします。L1正則化は特徴量の選択を促進し、いくつかの特徴量を重要視する傾向があります。α（alpha）は正則化の強度を調整するハイパーパラメータです。
# DecisionTreeRegressor: 決定木回帰は決定木をベースにした回帰モデルで、特徴量の値に基づいて目的変数を予測します。特徴としては、非線形な関係をキャプチャできる点があります。
# RandomForestRegressor: ランダムフォレスト回帰は複数の決定木を組み合わせたアンサンブルモデルです。複数の決定木を使用することで、過学習を抑制し、高い予測性能を提供します。特徴としては、非線形関係のモデリングに有効です。
# XGBRegressor: XGBoost回帰は勾配ブースティングモデルで、勾配ブースティングのアンサンブル技術を使用して、多数の決定木モデルを組み合わせて予測を行います。特徴としては、高い予測性能と汎化能力があります。
# LGBMRegressor: LightGBM回帰も勾配ブースティングモデルの一種で、LightGBMライブラリを使用します。高速で効率的な勾配ブースティングアルゴリズムを提供し、大規模なデータセットに適しています。高速で正確な予測が特徴です。

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import root_mean_squared_error

# 評価指標（コンペより引用）
# 提出物は、予測値の対数と観測された販売価格の対数との間のRMSE（Root-Mean-Squared-Error）で評価される。(対数をとるということは、高価な住宅と安価な住宅の予測誤差が同じように結果に影響することを意味する)。

# X: 説明変数、y: 目的変数
X = df_train.drop(["SalePrice"], axis=1)
y = df_train["SalePrice"]

# データ分割
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Pythonでは関数やオブジェクトをリストに入れることができます。ここでは、modelというリストに様々な回帰モデルのインスタンスを格納しています。
models = [
    LinearRegression(),
    Ridge(alpha=1.0),
    Lasso(alpha=1.0),
    DecisionTreeRegressor(),
    RandomForestRegressor(),
    XGBRegressor(),
    LGBMRegressor(),
]
# 結果を保存するためのデータフレームを作成しておく
results = pd.DataFrame(columns=["Model", "RMSE"])


for model in models:
    model_name = model.__class__.__name__
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    rmse = root_mean_squared_error(y_test, y_pred)
    results = pd.concat(
        [results, pd.DataFrame({"Model": [model_name], "RMSE": [rmse]})],
        ignore_index=True,
    )

results = results.sort_values(by="RMSE")

display(results)
print("なお、目的変数の各統計量(y.describe())は以下の通り:")
display(y.describe())

## モデルのチューニング（グリッドサーチ）

In [ ]:
# モデル選択にてリッジ回帰が一番RMSEが低い結果になったため、今度はRidge回帰の中でハイパーパラメータのalpha値を変えていき、RMSEが一番小さくなるalpha値を確認していく
# グリッドサーチでは、指定したハイパーパラメータの候補値を組み合わせて全ての組み合わせを試し、最適な組み合わせを見つける

from sklearn.model_selection import GridSearchCV

# 開始値（最小値）と終了値（最大値）を指定
start_alpha = 0.01
end_alpha = 100

# 対数スケールでalphaの値を20個生成
alphas = np.logspace(np.log10(start_alpha), np.log10(end_alpha), num=20)

ridge = Ridge()

# グリッドサーチのパラメータグリッドを定義
param_grid = {"alpha": alphas}

# クロスバリデーションとグリッドサーチを組み合わせて最適なalphaを探索
grid_search = GridSearchCV(
    ridge, param_grid, cv=5, scoring="neg_root_mean_squared_error"
)  # グリッドサーチの前準備
grid_search.fit(
    X_train, y_train
)  # ここでグリッドサーチを実行。各αを設定したモデルについて学習をさせて、どのαを使ったときに最良の結果が得られるのか調べている

best_alpha = grid_search.best_params_["alpha"]
best_score = grid_search.best_score_
print(f"Best alpha: {best_alpha}")
print(f"Best score(negRMSE): {best_score}")

# チューニング前と比べて、RMSEが減少している（0.117716→0.113794）
# このRMSEでRMSE/標準偏差を計算すると、=0.2847となりまあまあ小さい。つまりそこそこの精度がある

In [ ]:
# 最適なalphaでRidge回帰モデルを再訓練
model = Ridge(alpha=best_alpha).fit(X, y)

# テストデータに対して予測を行い、exp(x) - 1を適用して元のスケールに戻す
# 目的変数(SalePrice)の歪度調整のときに行った処理（data = np.log(df_train["SalePrice"])）によって、テストデータの予測結果SalePriceはlog対数が適用されている。そのため、予測結果に対して元のスケールに戻す処理が必要
# 疑問：対数変換するときに、log(y + 1)としていなかったのに、exp(x) - 1として良いのか？
sub_pred = np.expm1(model.predict(df_test))
submission = pd.DataFrame({"Id": df_test.index, "SalePrice": sub_pred})
submission.to_csv("train_test_submission/submission.csv", index=False)

In [ ]:
# memo:他にやりたいこと
# 特徴量重要度は？→ridge回帰だと正則化してるとかで？？

feature_names = X_train.columns
coefficients = abs(model.coef_)

df_model_coef = pd.DataFrame({"feature": feature_names, "coefficient": coefficients})
df_model_coef = df_model_coef.sort_values(by="coefficient", ascending=False)

display(df_model_coef.head(10))
plt.figure(figsize=(40, 10))
plt.bar(df_model_coef["feature"], df_model_coef["coefficient"])
plt.xticks(fontsize=10, rotation=90)
plt.show()